In [65]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

In [66]:
kospi = pd.read_excel("kospi100.xlsx", engine='openpyxl')

test = []
companies = kospi['종목명'][:2].tolist()
companies

C:\Users\ahn\anaconda3\envs\CapstonStockProject\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


['삼성전자', 'LG에너지솔루션']

In [67]:
five_dict = {}
ten_dict = {}
price_dict = {}

In [68]:
# 데이터 - CSV 파일 가져오기
for company in companies:
    csv_df = pd.read_csv(f'{company}.csv')  # CSV 파일명 입력
    

    

    csv_df = csv_df.loc[:, ['Date', 'Close', 'BPS', 'EPS', '금리', '환율']]
    csv_df = csv_df.set_index('Date') # 날짜를 index로 바꿈
    csv_df

    csv_df.dropna() # 결측치 처리

    # 날짜 빼고 정규화 작업 진행 -> MinMaxScaler 사용
    scaler = MinMaxScaler()

    # 정규화 수행 -> 정규화된 데이터프레임은 scaled_df
    scale_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
    scaled_df = scaler.fit_transform(csv_df[scale_cols]) # 정규화된 데이터는 넘파이 형태
    scaled_df = pd.DataFrame(scaled_df, columns = scale_cols) # Pandas DataFrame 형태로 변경

    # 테스트
    scaled_df

    # 날짜를 뺐으니 다시 날짜를 index 로 붙여줍니다
    scaled_csv_df = scaled_df.set_index(csv_df.index)
    scaled_csv_df

    # 입력데이터, 정답(예측하는)데이터를 정의
    # 입력데이터 -> 이전날까지의 종가와 변수들
    # 정답데이터 -> 다음날의 종가

    feature_cols = ['Close', 'BPS', 'EPS', '금리', '환율']
    label_cols = ['Close']

    # 입력데이터, 정답데이터 프레임 -> feature_df, label_df
    feature_df = scaled_csv_df[feature_cols]
    label_df = scaled_csv_df[label_cols]

    print(feature_df)
    print(label_df)


    # DataFrame을 Numpy 형태로 저장
    feature_np = feature_df.to_numpy()
    label_np = label_df.to_numpy()

    # sequence_dataset 만들기
    def make_sequence_dataset(feature, label, window_size):

        feature_list = []   # 생성될 feature list
        label_list = []     # 생성될 label list

        for i in range(len(feature)-window_size): # range는 전체값에서 window_size를 뺀 값

            feature_list.append(feature[i:i+window_size]) # feature list 에 i번째서 부터 window size 만큼의 입력데이터를 추가
            label_list.append(label[i+window_size]) # label list 에 그 다음 번째('window_size + 1' 번째)의 정답데이터를 추가

        return np.array(feature_list), np.array(label_list)







    ### **5일치 예측** -> 출력층 units수 = 5

    window_size = 80  # window_size 만큼의 입력데이터를 이용해 바로 다음 값에 오는 Close 값을 예측

    X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

    print(X.shape, y.shape)

    # 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
    split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

    # 훈련data 는 전체 데이터의 70퍼센트
    X_train = X[0:split]
    y_train = y[0:split]

    # 테스트data 는 전체 데이터의 30퍼센트
    X_test = X[split:]
    y_test = y[split:]

    # shape를 출력 -> 분할된 데이터 값으로 출력됨
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)

    #LSTM 모델 구축
    model = Sequential()

    # 1차원 feature map 생성
    model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

    # LSTM layer
    model.add(LSTM(units = 16, activation = 'tanh'))
    model.add(Dense(units = 16, activation = 'sigmoid'))
    model.add(Dense(units = 5)) # 출력층 -> 출력층의 unit 수를 조절해서 5일치, 10일치, 20일치, 30일치 등등 예측이 가능!
    model.summary()

    # 모델 컴파일
    loss = Huber()
    optimizer = Adam(0.0005)
    model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

    # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
    earlystopping = EarlyStopping(monitor='loss', patience=10)

    # 모델 학습 -> epoch은 100번 진행
    model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

    # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
    predictions = model.predict(X_test)

    #예측 비교하기
    print(predictions)

    print(y_test) #predictions 와 대충 값이 일치해야함

    # 실제값 변환
    # 실제값으로 변화시키기 위해서 기존 데이터에서 종가의 최댓값과 최솟값을 가져옵니다
    close_min = csv_df['Close'].min()
    close_max = csv_df['Close'].max()

    print(csv_df['Close'].min())
    print(csv_df['Close'].max())

    # MinMaxScaler이용해서 실제값으로 역변환
    scaler2 = MinMaxScaler()
    scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

    # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
    scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
    scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

    # 예측한 출력값을 실제값으로 역변환
    original_pred_values = scaler2.inverse_transform(predictions)

    # 역변환된 예측값 출력
    original_pred_values

    # 테스트 data의 실제값 -> 정규화시켰던 값을 split했었기 때문에 원본 데이터에서 다시 분리해서 가져옴

    # original_label_df -> 데이터셋의 실제 종가값
    original_label_df = csv_df[label_cols]

    # 다시 테스트용만큼의 값으로 분리
    original_y_train = original_label_df[0:split]
    original_y_test = original_label_df[split:]
    original_y_test = original_y_test[window_size:] # window_size만큼의 데이터 제거(예측에 사용하였으므로)

    # 역변환된 실제값 출력(종가)
    original_y_test

    # 실제로 에측하기
    # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
    pred_feature = feature_df.tail(window_size)

    pred_feature

    pred_feature_list = []
    pred_feature_list.append(pred_feature)
    pred_feature = np.array(pred_feature_list)
    print(pred_feature.shape)

    # 5일 예측 -> 5일까지의 예측값 5개 출력
    predictions_5d = model.predict(pred_feature)
    pred_values_5d = scaler2.inverse_transform(predictions_5d)
    print(pred_values_5d)
    print(len(pred_values_5d[0]))

    # 예측값 추가해서 DataFrame으로 만들기
    # 2020년 부터 현재까지의 종가값을 제시하고 앞에 5일 예측값을 추가
    pred5dCsvDf = csv_df[['Close']].loc['2020-01-01':,:]

    for i in range (len(pred_values_5d[0])):
        pred5dCsvDf.loc[(i+1),:]=[pred_values_5d[0,i]]

    pred5dCsvDf

    # DataFrame을 csv파일로 만들기
    pred5dCsvDf.to_csv('5d_predict_filename.csv', header=False, index=True)





    ### **10일치 예측**

    window_size = 80

    X, y = make_sequence_dataset(feature_np, label_np, window_size) # X에는 np.array(feature_list), y에는 np.array(label_list) 가 대입됨

    print(X.shape, y.shape)

    # 모델 훈련을 위한 준비 -> 훈련을 하고, 제대로 훈련이 됐는지 테스트
    split = int(len(X)*0.7) # 테스트 데이터로 분리 -> train:test = 7:3

    # 훈련data 는 전체 데이터의 70퍼센트
    X_train = X[0:split]
    y_train = y[0:split]

    # 테스트data 는 전체 데이터의 30퍼센트
    X_test = X[split:]
    y_test = y[split:]

    # shape를 출력 -> 분할된 데이터 값으로 출력됨
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)

    #LSTM 모델 구축
    model = Sequential()

    # 1차원 feature map 생성
    model.add(Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[window_size, 5]))# input_shape = (40,5) -> 다음값 예측을 위한 이전 40개(window_size)의 값과 5개의 특성을 입력으로 넣습니다

    # LSTM layer
    model.add(LSTM(units = 16, activation = 'tanh'))
    model.add(Dense(units = 16, activation = 'sigmoid'))
    model.add(Dense(units = 10)) # 출력층 -> 출력층의 unit 수를 조절해서 5일치, 10일치, 20일치, 30일치 등등 예측이 가능!
    model.summary()

    # 모델 컴파일
    from tensorflow.keras.losses import Huber
    from tensorflow.keras.optimizers import Adam
    
    loss = Huber()
    optimizer = Adam(0.0005)
    model.compile(loss=loss, optimizer=optimizer, metrics=['mse']) # 손실 함수는 Huber, 옵티마이저는 Adam,  평가지표는 mse로 설정

    # 조기종료 설정 -> earlystopping은 10번의 epoch통안 loss 개선이 없다면 학습을 멈춤
    earlystopping = EarlyStopping(monitor='loss', patience=10)

    # 모델 학습 -> epoch은 100번 진행
    model.fit(X_train, y_train, epochs=100, batch_size=32, callbacks=[earlystopping])

    # 주가 예측 -> test 데이터를 이용하여 학습된 LSTM모델을 테스트
    predictions = model.predict(X_test)

    #예측 비교하기
    print(predictions)

    print(y_test) #predictions 와 대충 값이 일치해야함

    # MinMaxScaler이용해서 실제값으로 역변환
    scaler2 = MinMaxScaler()
    scaled_df2 = scaler2.fit_transform(csv_df[['Close']])

    # MinMaxScaler에 정규화에 사용한 최솟값과 최댓값을 설정
    scaler2.data_min_ = close_min  # 정규화에 사용한 최솟값
    scaler2.data_max_ = close_max  # 정규화에 사용한 최댓값

    # 예측한 출력값을 실제값으로 역변환
    original_pred_values = scaler2.inverse_transform(predictions)

    # 역변환된 예측값 출력
    original_pred_values

    # 테스트 data의 실제값 -> 정규화시켰던 값을 split했었기 때문에 원본 데이터에서 다시 분리해서 가져옴

    # original_label_df -> 데이터셋의 실제 종가값
    original_label_df = csv_df[label_cols]

    # 다시 테스트용만큼의 값으로 분리
    original_y_train = original_label_df[0:split]
    original_y_test = original_label_df[split:]
    original_y_test = original_y_test[window_size:] # window_size만큼의 데이터 제거(예측에 사용하였으므로)

    # 역변환된 실제값 출력(종가)
    original_y_test

    # 실제로 에측하기
    # feature_df 에서 가장 최근의 값을 window_size 만큼 가져옴
    pred_feature = feature_df.tail(window_size)

    pred_feature

    pred_feature_list = []
    pred_feature_list.append(pred_feature)
    pred_feature = np.array(pred_feature_list)
    print(pred_feature.shape)

    # 10일 예측 -> 10일까지의 예측값 10개 출력
    predictions_10d = model.predict(pred_feature)
    pred_values_10d = scaler2.inverse_transform(predictions_10d)
    print(pred_values_10d)
    print(len(pred_values_10d[0]))

    # 예측값 추가해서 CSV파일로 만들기
    # 2020년 부터 현재까지의 종가값을 제시하고 앞에 10일 예측값을 추가
    pred10dCsvDf = csv_df[['Close']].loc['2020-01-01':,:]

    for i in range (len(pred_values_10d[0])):
        pred10dCsvDf.loc[(i+1),:]=[pred_values_10d[0,i]]

    pred10dCsvDf

    pred10dCsvDf.to_csv('10d_predict_filename.csv', header=False, index=True)


    #현재 종가값만 따로 csv파일로 저장

    predNowCsvDf = csv_df[['Close']].loc['2020-01-01':,:]

    predNowCsvDf.to_csv('Data_filename.csv', header=False, index=True)

    pred5dCsvDf = pred5dCsvDf.reset_index()
    day_five = pred5dCsvDf.values
    day_five = day_five.tolist()
    day_five

    pred10dCsvDf = pred10dCsvDf.reset_index()
    day_ten = pred10dCsvDf.values
    day_ten = day_ten.tolist()
    day_ten
    
    five_dict[company] = day_five
    ten_dict[company] = day_ten



               Close       BPS       EPS   금리        환율
Date                                                   
2015-01-02  0.075510  0.595634  0.656130  0.5  0.102171
2015-01-05  0.076371  0.595634  0.656130  0.5  0.146760
2015-01-06  0.065461  0.595634  0.656130  0.5  0.140353
2015-01-07  0.068906  0.595634  0.656130  0.5  0.111652
2015-01-08  0.070916  0.595634  0.656130  0.5  0.112934
...              ...       ...       ...  ...       ...
2023-08-02  0.697100  0.021548  0.016485  1.0  0.603029
2023-08-03  0.681309  0.021548  0.016485  1.0  0.619225
2023-08-04  0.674131  0.021548  0.016485  1.0  0.621864
2023-08-07  0.677003  0.021548  0.016485  1.0  0.638905
2023-08-08  0.664083  0.021548  0.016485  1.0  0.650950

[2119 rows x 5 columns]
               Close
Date                
2015-01-02  0.075510
2015-01-05  0.076371
2015-01-06  0.065461
2015-01-07  0.068906
2015-01-08  0.070916
...              ...
2023-08-02  0.697100
2023-08-03  0.681309
2023-08-04  0.674131
2023-08-07  0.67

45/45 [==============================] - 1s 11ms/step - loss: 1.8974e-04 - mse: 3.7947e-04
Epoch 62/100
45/45 [==============================] - 1s 11ms/step - loss: 1.9136e-04 - mse: 3.8273e-04
Epoch 63/100
45/45 [==============================] - 1s 11ms/step - loss: 1.9207e-04 - mse: 3.8413e-04
Epoch 64/100
45/45 [==============================] - 1s 11ms/step - loss: 1.9345e-04 - mse: 3.8689e-04
Epoch 65/100
45/45 [==============================] - 1s 11ms/step - loss: 1.8698e-04 - mse: 3.7397e-04
Epoch 66/100
45/45 [==============================] - 1s 11ms/step - loss: 1.8575e-04 - mse: 3.7150e-04
Epoch 67/100
45/45 [==============================] - 1s 11ms/step - loss: 1.7929e-04 - mse: 3.5858e-04
Epoch 68/100
45/45 [==============================] - 1s 11ms/step - loss: 1.8206e-04 - mse: 3.6412e-04
Epoch 69/100
45/45 [==============================] - 1s 12ms/step - loss: 1.8840e-04 - mse: 3.7681e-04
Epoch 70/100
45/45 [==============================] - 1s 11ms/step - loss: 1.

1/1 [==============================] - 0s 15ms/step
[[64017.707 66172.49  62688.266 64169.703 65679.53 ]]
5
(2039, 80, 5) (2039, 1)
(1427, 80, 5) (1427, 1)
(612, 80, 5) (612, 1)
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 80, 32)            832       
                                                                 
 lstm_9 (LSTM)               (None, 16)                3136      
                                                                 
 dense_18 (Dense)            (None, 16)                272       
                                                                 
 dense_19 (Dense)            (None, 10)                170       
                                                                 
Total params: 4,410
Trainable params: 4,410
Non-trainable params: 0
_________________________________________________________________
Epoch 

45/45 [==============================] - 1s 11ms/step - loss: 1.7570e-04 - mse: 3.5140e-04
Epoch 71/100
45/45 [==============================] - 1s 11ms/step - loss: 1.6859e-04 - mse: 3.3718e-04
Epoch 72/100
45/45 [==============================] - 1s 11ms/step - loss: 1.9043e-04 - mse: 3.8086e-04
Epoch 73/100
45/45 [==============================] - 1s 11ms/step - loss: 2.0330e-04 - mse: 4.0661e-04
Epoch 74/100
45/45 [==============================] - 1s 11ms/step - loss: 1.7340e-04 - mse: 3.4679e-04
Epoch 75/100
45/45 [==============================] - 1s 11ms/step - loss: 1.6334e-04 - mse: 3.2668e-04
Epoch 76/100
45/45 [==============================] - 1s 12ms/step - loss: 1.7446e-04 - mse: 3.4892e-04
Epoch 77/100
45/45 [==============================] - 1s 12ms/step - loss: 1.6591e-04 - mse: 3.3183e-04
Epoch 78/100
45/45 [==============================] - 1s 12ms/step - loss: 1.6571e-04 - mse: 3.3142e-04
Epoch 79/100
45/45 [==============================] - 1s 12ms/step - loss: 1.

1/1 [==============================] - 0s 16ms/step
[[64608.023 63259.86  63884.047 64842.074 64365.996 64272.562 63184.336
  64066.2   65535.125 64214.83 ]]
10
               Close  BPS       EPS   금리        환율
Date                                              
2022-01-27  0.555970  0.0  0.000000  0.0  0.032716
2022-01-28  0.350746  0.0  0.000000  0.0  0.043965
2022-02-03  0.451493  0.0  0.000000  0.0  0.030052
2022-02-04  0.552239  0.0  0.000000  0.0  0.031523
2022-02-07  0.716418  0.0  0.000000  0.0  0.018643
...              ...  ...       ...  ...       ...
2023-08-02  0.701493  1.0  0.834217  1.0  0.384203
2023-08-03  0.697761  1.0  0.834217  1.0  0.409326
2023-08-04  0.686567  1.0  0.834217  1.0  0.413420
2023-08-07  0.589552  1.0  0.834217  1.0  0.439856
2023-08-08  0.630597  1.0  0.834217  1.0  0.458539

[378 rows x 5 columns]
               Close
Date                
2022-01-27  0.555970
2022-01-28  0.350746
2022-02-03  0.451493
2022-02-04  0.552239
2022-02-07  0.716418
...  

7/7 [==============================] - 0s 13ms/step - loss: 0.0021 - mse: 0.0041
Epoch 66/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0020 - mse: 0.0041
Epoch 67/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0019 - mse: 0.0039
Epoch 68/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0019 - mse: 0.0037
Epoch 69/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0018 - mse: 0.0037
Epoch 70/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0018 - mse: 0.0036
Epoch 71/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0017 - mse: 0.0035
Epoch 72/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0017 - mse: 0.0035
Epoch 73/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0017 - mse: 0.0034
Epoch 74/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0017 - mse: 0.0034
Epoch 75/100
7/7 [==============================] - 0s 13ms/step - loss: 

1/1 [==============================] - 0s 16ms/step
[[518793.   493050.62 520409.7  523930.34 510335.94]]
5
(298, 80, 5) (298, 1)
(208, 80, 5) (208, 1)
(90, 80, 5) (90, 1)
Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_11 (Conv1D)          (None, 80, 32)            832       
                                                                 
 lstm_11 (LSTM)              (None, 16)                3136      
                                                                 
 dense_22 (Dense)            (None, 16)                272       
                                                                 
 dense_23 (Dense)            (None, 10)                170       
                                                                 
Total params: 4,410
Trainable params: 4,410
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100

7/7 [==============================] - 0s 12ms/step - loss: 0.0124 - mse: 0.0247
Epoch 77/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0119 - mse: 0.0238
Epoch 78/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0115 - mse: 0.0231
Epoch 79/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0109 - mse: 0.0219
Epoch 80/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0104 - mse: 0.0209
Epoch 81/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0100 - mse: 0.0201
Epoch 82/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0098 - mse: 0.0195
Epoch 83/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0094 - mse: 0.0188
Epoch 84/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0090 - mse: 0.0179
Epoch 85/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0088 - mse: 0.0177
Epoch 86/100
7/7 [==============================] - 0s 12ms/step - loss: 

1/1 [==============================] - 0s 16ms/step
[[490531.5  471671.03 490387.4  485367.25 474636.6  496256.5  494484.5
  498854.4  484677.7  475626.2 ]]
10


In [69]:
five_dict

{'삼성전자': [['2020-01-02', 55200.0],
  ['2020-01-03', 55500.0],
  ['2020-01-06', 55500.0],
  ['2020-01-07', 55800.0],
  ['2020-01-08', 56800.0],
  ['2020-01-09', 58600.0],
  ['2020-01-10', 59500.0],
  ['2020-01-13', 60000.0],
  ['2020-01-14', 60000.0],
  ['2020-01-15', 59000.0],
  ['2020-01-16', 60700.0],
  ['2020-01-17', 61300.0],
  ['2020-01-20', 62400.0],
  ['2020-01-21', 61400.0],
  ['2020-01-22', 62300.0],
  ['2020-01-23', 60800.0],
  ['2020-01-28', 58800.0],
  ['2020-01-29', 59100.0],
  ['2020-01-30', 57200.0],
  ['2020-01-31', 56400.0],
  ['2020-02-03', 57200.0],
  ['2020-02-04', 58900.0],
  ['2020-02-05', 59500.0],
  ['2020-02-06', 61100.0],
  ['2020-02-07', 60400.0],
  ['2020-02-10', 59700.0],
  ['2020-02-11', 59900.0],
  ['2020-02-12', 60500.0],
  ['2020-02-13', 60700.0],
  ['2020-02-14', 61800.0],
  ['2020-02-17', 61500.0],
  ['2020-02-18', 59800.0],
  ['2020-02-19', 60200.0],
  ['2020-02-20', 60000.0],
  ['2020-02-21', 59200.0],
  ['2020-02-24', 56800.0],
  ['2020-02-25', 579

In [70]:
ten_dict

{'삼성전자': [['2020-01-02', 55200.0],
  ['2020-01-03', 55500.0],
  ['2020-01-06', 55500.0],
  ['2020-01-07', 55800.0],
  ['2020-01-08', 56800.0],
  ['2020-01-09', 58600.0],
  ['2020-01-10', 59500.0],
  ['2020-01-13', 60000.0],
  ['2020-01-14', 60000.0],
  ['2020-01-15', 59000.0],
  ['2020-01-16', 60700.0],
  ['2020-01-17', 61300.0],
  ['2020-01-20', 62400.0],
  ['2020-01-21', 61400.0],
  ['2020-01-22', 62300.0],
  ['2020-01-23', 60800.0],
  ['2020-01-28', 58800.0],
  ['2020-01-29', 59100.0],
  ['2020-01-30', 57200.0],
  ['2020-01-31', 56400.0],
  ['2020-02-03', 57200.0],
  ['2020-02-04', 58900.0],
  ['2020-02-05', 59500.0],
  ['2020-02-06', 61100.0],
  ['2020-02-07', 60400.0],
  ['2020-02-10', 59700.0],
  ['2020-02-11', 59900.0],
  ['2020-02-12', 60500.0],
  ['2020-02-13', 60700.0],
  ['2020-02-14', 61800.0],
  ['2020-02-17', 61500.0],
  ['2020-02-18', 59800.0],
  ['2020-02-19', 60200.0],
  ['2020-02-20', 60000.0],
  ['2020-02-21', 59200.0],
  ['2020-02-24', 56800.0],
  ['2020-02-25', 579

In [ ]:
import requests

data = {
    'day_five': day_five,
    'day_ten': day_ten
}
# url 달라 선빈님
server_url = 'http://127.0.0.1:5000/sentiment'

response = requests.post(server_url, json=data)  # POST 요청으로 변경, 헤더는 자동으로 설정됨

if response.status_code == 200:
    print('성공')
else:
    print('실패:', response.status_code)